In [ ]:
#install required packages
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import torch

# path exports

In [ ]:
sample = 'cosmic_michel_f3/aa'
base_path = '/mnt/data1/tkar/cosmic_new/'
base_path_DKarres = '/mnt/data1/karres/cosmic_test/'
dir_cm_new_fStore = base_path + 'feature_store/'  + sample + '/'

results_path = base_path + 'results/' 
pcle_results_path = results_path + 'particle_level/'
edge_results_path = results_path + 'edge_level/'
track_results_path = results_path + 'track_level/'

cosmic_dir = '/mnt/data1/karres/cosmics_test/feature_store_cosmic/trainset/'
michel_dir = '/mnt/data1/karres/cosmics_test/feature_store_michel/trainset/'
cosmic_michel_dir = '/mnt/data1/karres/cosmics_test/feature_store_cosmic_michel/trainset/' 

# Control Plots

### function defs

In [ ]:
#Control plots:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob

#Hit/particle level plots:

def get_hit_data(dir, samplesize, key, filter_particle=None):
    #Iterate through all truth and particle csvs from the directory
    histogram_data = []
    i = 1

    for truth, particles in zip(glob.glob(dir+'/*truth.csv'), glob.glob(dir+'/*particles.csv')):
        truth_df = pd.read_csv(truth)
        particle_df = pd.read_csv(particles)

        if filter_particle == 'muon':
            particle_df = particle_df[(particle_df['particle_type'] == 13) | (particle_df['particle_type'] == -13)]
        elif filter_particle == 'electron':
            particle_df = particle_df[(particle_df['particle_type'] == 11) | (particle_df['particle_type'] == -11)]
        
        if key in particle_df.columns:
            histogram_data.extend(particle_df[key].tolist())
        elif key in truth_df.columns:
            histogram_data.extend(truth_df[key].tolist())
        else:
            print('Key not found in CSV files')

        i += 1
        if i > samplesize:
            break
    
    return histogram_data

#Position plots
def get_position_data(dir, samplesize):
    #cartesian
    x_data = get_hit_data(dir, samplesize, 'x')
    y_data = get_hit_data(dir, samplesize, 'y')
    z_data = get_hit_data(dir, samplesize, 'z')
    
    #spherical
    r = np.sqrt(np.array(x_data)**2 + np.array(y_data)**2+ np.array(z_data)**2)
    phi = np.arctan2(np.array(z_data), np.array(x_data))
    theta = np.arccos(np.array(y_data)/r)
    
    #cylindrical
    rho = np.sqrt(np.array(x_data)**2 + np.array(y_data)**2)
    phi = np.arctan2(y_data, x_data)
    z_cyl = z_data
    
    return x_data, y_data, z_data, rho, phi, z_cyl

def get_particle_data(dir, samplesize):
    pid = get_hit_data(dir, samplesize, 'particle_type')

    pt_mu = get_hit_data(dir, samplesize, 'pt', 'muon')
    pt_e = get_hit_data(dir, samplesize, 'pt', 'electron')

    nhits_mu = get_hit_data(dir, samplesize, 'nhits', 'muon')
    nhits_e = get_hit_data(dir, samplesize, 'nhits', 'electron')
    
    radius_mu = get_hit_data(dir, samplesize, 'radius', 'muon')
    radius_e = get_hit_data(dir, samplesize, 'radius', 'electron')

    vx_mu = get_hit_data(dir, samplesize, 'vx', 'muon')
    vy_mu = get_hit_data(dir, samplesize, 'vy', 'muon')
    vz_mu = get_hit_data(dir, samplesize, 'vz', 'muon')

    vx_e = get_hit_data(dir, samplesize, 'vx', 'electron')
    vy_e = get_hit_data(dir, samplesize, 'vy', 'electron')
    vz_e = get_hit_data(dir, samplesize, 'vz', 'electron')

    px_mu = get_hit_data(dir, samplesize, 'px', 'muon')
    py_mu = get_hit_data(dir, samplesize, 'py', 'muon')
    pz_mu = get_hit_data(dir, samplesize, 'pz', 'muon')

    px_e = get_hit_data(dir, samplesize, 'px', 'electron')
    py_e = get_hit_data(dir, samplesize, 'py', 'electron')
    pz_e = get_hit_data(dir, samplesize, 'pz', 'electron')

    p_tot_mu = np.sqrt(np.array(px_mu)**2 + np.array(py_mu)**2 + np.array(pz_mu)**2)
    p_tot_e = np.sqrt(np.array(px_e)**2 + np.array(py_e)**2 + np.array(pz_e)**2)
    
    pt = [pt_mu, pt_e]
    nhits = [nhits_mu, nhits_e]
    radius = [radius_mu, radius_e]
    v_mu = [vx_mu, vy_mu, vz_mu]
    v_e = [vx_e, vy_e, vz_e]
    p_mu = [px_mu, py_mu, pz_mu]
    p_e = [px_e, py_e, pz_e]
    p_tot = [p_tot_mu, p_tot_e]

    return pid, pt, nhits, radius, v_mu, v_e, p_mu, p_e, p_tot

def get_graph_data(dir, samplesize):
    num_edges = []
    num_nodes = []

    for graph in glob.glob(dir+'/*.pyg'):
        graph = torch.load(graph)
        num_edges.append(graph.edge_index.shape[1])
        num_nodes.append(graph.num_nodes)

    
    return num_edges, num_nodes

### Position Plots

In [ ]:
plot_file_name = 'position_plots_c_m_cm.pdf'

x_c, y_c, z_c, r_c, phi_c, eta_c = get_position_data(cosmic_dir, 10000)
x_m, y_m, z_m, r_m, phi_m, eta_m = get_position_data(michel_dir, 10000)
#x_cm, y_cm, z_cm, r_cm, phi_cm, eta_cm = get_position_data(cosmic_michel_dir, 10000)

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

axs[0,0].hist(x_c, bins=50, linewidth=1.5, edgecolor='black', histtype='step', label='Cosmics only')
axs[0,0].hist(x_m, bins=50, linewidth=1.5, edgecolor='red', histtype='step', label='Michel only')
#axs[0,0].hist(x_cm, bins=50, linewidth=1.5, edgecolor='blue', histtype='step', label='Cosmics with Michel')
axs[0,0].set_xlabel(r'$x$ [mm]')
axs[0,0].set_title(r'Distribution of $x$ positions')

axs[0,1].hist(y_c, bins=50, linewidth=1.5, edgecolor='black', histtype='step')
axs[0,1].hist(y_m, bins=50, linewidth=1.5, edgecolor='red', histtype='step')
#axs[0,1].hist(y_cm, bins=50, linewidth=1.5, edgecolor='blue', histtype='step')
axs[0,1].set_xlabel(r'$y$ [mm]')
axs[0,1].set_title(r'Distribution of $y$ positions')

axs[0,2].hist(z_c, bins=50, linewidth=1.5, edgecolor='black', histtype='step')
axs[0,2].hist(z_m, bins=50, linewidth=1.5, edgecolor='red', histtype='step')
#axs[0,2].hist(z_cm, bins=50, linewidth=1.5, edgecolor='blue', histtype='step')
axs[0,2].set_xlabel(r'$z$ [mm]')
axs[0,2].set_title(r'Distribution of $z$ positions')

axs[1,0].hist(r_c, bins=50, linewidth=1.5, edgecolor='black', histtype='step')
axs[1,0].hist(r_m, bins=50, linewidth=1.5, edgecolor='red', histtype='step')
#axs[1,0].hist(r_cm, bins=50, linewidth=1.5, edgecolor='blue', histtype='step')
axs[1,0].set_xlabel(r'$\rho$ [mm]')
axs[1,0].set_title(r'Distribution of radial distances $\rho$')

axs[1,1].hist(phi_c, bins=50, linewidth=1.5, edgecolor='black', histtype='step')
axs[1,1].hist(phi_m, bins=50, linewidth=1.5, edgecolor='red', histtype='step')
#axs[1,1].hist(phi_cm, bins=50, linewidth=1.5, edgecolor='blue', histtype='step')
axs[1,1].set_xlabel(r'$\varphi$ [rad]')
axs[1,1].set_title(r'Distribution of azimuthal angles $\varphi$')

axs[1,2].hist(eta_c, bins=50, linewidth=1.5, edgecolor='black', histtype='step')
axs[1,2].hist(eta_m, bins=50, linewidth=1.5, edgecolor='red', histtype='step')
#axs[1,2].hist(eta_cm, bins=50, linewidth=1.5, edgecolor='blue', histtype='step')
axs[1,2].set_xlabel(r'$z$')
axs[1,2].set_title(r'Distribution of $z$ positions')

for ax in axs.flat:
    ax.set_ylabel('Frequency')
    ax.set_yscale('log')
    
fig.legend(loc='upper center', ncol=2, bbox_to_anchor=(0.5, 0.95))
plt.tight_layout(rect=[0, 0, 1, 0.91])
fig.suptitle('Hit Position Distributions of 10000 Events per Dataset in Cartesian and Cylindrical Coordinates', fontsize=16)
fig.savefig(pcle_results_path + plot_file_name, format="pdf", bbox_inches="tight")
plt.show()

### Momentum Plots